In [1]:
import os, sys
import pandas as pd
import numpy as np
from functools import reduce

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from Scripts import Data_Loader_Functions as dL
from Scripts import Results_Evaluation as rE
RESULTS = os.path.join(module_path, 'Results', 'Thesis')
DATA = os.path.join(module_path, 'Data', 'Augmented Data')

In [64]:
# Create DF with TP TN FP FN
df = pd.read_csv(os.path.join(module_path, "BCDL_all.csv"))

In [36]:
# Load Pivot & Subjects
pivot = dL.create_pivot(os.path.join(DATA, 'group_2'), 'Session', 'Person', 'Session')
subjects = dL.create_pain_df(os.path.join(DATA, 'group_2'))['Person'].unique()

In [37]:
def filter_granular_results(df, subjects, pivot, model_name):
    
    # Add columns to df [TP, TN, FP, FN]
    df['label'] = np.minimum(df['True Label'], 1)
    df['prediction'] = df[model_name].round()
    df['TP'] = (df['label'] == 1) & (df['prediction'] == 1)
    df['TN'] = (df['label'] == 0) & (df['prediction'] == 0)
    df['FP'] = (df['label'] == 0) & (df['prediction'] == 1)
    df['FN'] = (df['label'] == 1) & (df['prediction'] == 0)

    # Prepare pivot to be merged with df
    columns = np.insert(subjects.astype(object), 0, 'Session')
    pivot = pivot.reset_index()[columns].drop(0).reset_index(drop=True)
    pivot_has_positive = pivot[pivot.columns[1:]] != ''
    pivot_has_positive['Session'] = pivot['Session']
    pivot_has_positive = pivot_has_positive.drop(9)
    unpivot_has_positive = pivot_has_positive.melt(id_vars=['Session'])
    unpivot_has_positive = unpivot_has_positive.rename(columns={'Person': 'Subject ID', 'value': 'has_positive'})

    # Merge pivot and df to indicate, which subjects have positive examples in a given session
    df_merge = df.merge(unpivot_has_positive)
    
    # Return only those session/subject combinations that have positive examples
    return df_merge[df_merge['has_positive']]

In [65]:
df_merge_has_positive = filter_granular_results(df, subjects, pivot, 'BCDL')

In [66]:
df_test = df_merge_has_positive[(df_merge_has_positive['Subject ID'] == 59) & (df_merge_has_positive['Seed'] == 123)]

In [67]:
print("TP: ", df_test['TP'].sum())
print("TN: ", df_test['TN'].sum())
print("FP: ", df_test['FP'].sum())
print("FN: ", df_test['FN'].sum())
print("ACC: ", (df_test['TP'].sum() + df_test['TN'].sum()) / len(df_test))

TP:  469
TN:  252
FP:  688
FN:  63
ACC:  0.4898097826086957


In [41]:
df_1 = pd.read_csv(os.path.join(module_path, 'Results/Thesis/NEW/123 - Seed 123 - Sessions NEW/2020-01-25-234643_PAIN_4-sessions-Federated-central-pre-training_TEST.csv'))
rE.weighted_mean_SD(df_1, subjects, 'accuracy', False, pivot)

,43,48,52,59,64,80,92,96,107,109,115,120,Weighted Mean,Weighted SD
0,0.694611,0.789506,0.919086,0.48981,0.912587,0.624392,0.696678,0.835106,0.728125,0.787677,0.786175,0.631481,0.760872,0.121167


In [44]:
print("TP: ", df_1['subject_59_true_positives'][0])
print("TN: ", df_1['subject_59_true_negatives'][0])
print("FP: ", df_1['subject_59_false_positives'][0])
print("FN: ", df_1['subject_59_false_negatives'][0])

TP:  469.0
TN:  252.0
FP:  688.0
FN:  63.0


In [33]:
sum((469.0, 252, 688, 63))

1472.0

In [34]:
sum((488, 155, 785, 44))

1472

In [46]:
df_1 = pd.read_csv(os.path.join(module_path, 'Results/Thesis/NEW/123 - Seed 123 - Sessions NEW/2020-01-26-010842_PAIN_0-sessions-centralized-pre-training_TEST.csv'))
df_2 = pd.read_csv(os.path.join(module_path, 'Results/Thesis/NEW/123 - Seed 123 - Sessions NEW/2020-01-25-231525_PAIN_2-sessions-Centralized-pre-training_TEST.csv'))
df_3 = pd.read_csv(os.path.join(module_path, 'Results/Thesis/NEW/123 - Seed 123 - Sessions NEW/2020-01-25-234643_PAIN_4-sessions-Federated-central-pre-training_TEST.csv'))
df_4 = pd.read_csv(os.path.join(module_path, 'Results/Thesis/NEW/123 - Seed 123 - Sessions NEW/2020-01-26-003549_PAIN_7-sessions-Federated-central-pre-training-personalization_TEST.csv'))
df_5 = pd.read_csv(os.path.join(module_path, 'Results/Thesis/NEW/123 - Seed 123 - Sessions NEW/2020-01-26-010705_PAIN_10-sessions-Federated-central-pre-training-local-models_TEST.csv'))

In [72]:
df_1['subject_59_true_positives']

0    469.0
1      NaN
2      NaN
3      NaN
4      NaN
5      NaN
6      NaN
7      NaN
8      NaN
Name: subject_59_true_positives, dtype: float64

In [73]:
df_2['subject_59_true_positives']

0    469.0
1      NaN
2      NaN
3      NaN
4      NaN
5      NaN
6      NaN
7      NaN
8      NaN
Name: subject_59_true_positives, dtype: float64

In [74]:
df_3['subject_59_true_positives']

0    469.0
1      NaN
2      NaN
3      NaN
4      NaN
5      NaN
6      NaN
7      NaN
8      NaN
Name: subject_59_true_positives, dtype: float64

In [68]:
pivot

Person,43,48,52,59,64,80,92,96,107,109,115,120,# of Persons,Pain,No Pain,Pain %
Session,,,,,,,,,,,,,,,,
0,140,,72,,244,1052,464,,32,,60,116,12,2180,12424,0.15
1,,148,,532,64,536,696,,,600,,,12,2576,8792,0.23
2,,,,,64,484,,,848,,220,,11,1616,8756,0.16
3,,,,,,484,,,60,,56,188,11,788,7696,0.09
4,228,,,,248,660,724,112,828,,60,,11,2860,7276,0.28
5,,188,,,,792,,,,116,,,9,1096,7380,0.13
6,,,44,,,264,,512,,,,,8,820,4852,0.14
7,,,,,,,,,,,,,6,,3788,
8,,,120,,,,,88,,,,,3,208,3200,0.06
